# The North Remembers Project

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import re
import random
from pprint import pprint
from time import sleep
import math
from itertools import groupby
from collections import OrderedDict as odict
from pprint import pprint



from website.zhihu import zhihu_detect_with_client
from website.zhihu import zhihu_detect
from website.zhihu import get_comments_api_v4
from website.zhihu import get_valuable_conversations_api_v4
from website.zhihu import fetch_zhihu_answer
from website.zhihu import fetch_zhihu_article
import tools

from zhihu_oauth import ZhihuClient

TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)

### 取舍

数据存储用 github, 或者 bitbucket


提供导出为 rss, epub, pdf, github 自带检查历史的功能

不追求实效性



外链图片不作为重点
外链图片和视频链接, url 链接, 一并修复, 换行字符串等也一并处理, 七牛云?
md 中的 Latex 和 代码检测也在这里修复, pygment

### 调用关系





#### Collector

处理用户发出的命令

可以是 zhihu answer url, 问题页, 收藏夹页, 需要带有过滤条件
用户可能在过滤条件后再做修补删减

需要转为具体的爬虫任务, 并创建文件夹
单页收集到特定的文件夹, 否则收集到一个自动命名的文件夹

在文件夹下生成task

Resources
global_config
task_config (scanlist)


Collector.append(url, options)



#### Watcher



任务调度, 监视文件夹, 根据 task.yaml 抓取页面

task.yaml 也记录历史

task_type: single_page, 
update_batch: 10


track_new: true


通过扫描已有的待抓取任务, 在手动触发, 达到特定时间后, 分派抓取任务

    Watcher
        .watch()
        .remember()
        .status()
        .load_config_yaml
        .save_config_yaml
        .add_task add_tasks



Task

被调用抓取单一页面内容, 或者抓取一个 url 列表
抓取单一页面内容将存为 md, 不存 html

    Task
        ZhihuAnswerPageTask
        ZhihuAnswerIndexTask
        ZhihuColumnPageTask
        ZhihuColumnIndexTask

Task 会使用 Page.save / load

    .last_page()
    .fetch()
    .save() 传递一些参数给 page 如 watch_date
    .to_yaml_text()
    
    .priority
    .is_page_type
    .should_fetch
    .content_is_changed



ZhihuColumnPage
ZhihuColumnLister


Fetcher.request(url) => dict()? Page()

Page.write to / load from disk

PageTask.run => Fetcher.request(url) => New Page
PageTask.update => New Page.write
IndexTask.run => Fetcher.request(url) => [New Tasks]


Watcher -> PageTask -> Page (tmpl, fetcherAPI)
                            ZhihuColumnPage
                            ZhihuAnswerPage
                            WeixinArticlePage


Watcher -> IndexTask -> append more tasks
                        ZhihuColumnIndex
    





#### Fetcher

Page

页面内容分为 元数据 + 文章主体 + 评论, 评论先放放


Page.load()  返回一个读盘的页面

以md文件为单位

    ZhihuColumnPage
        .load(path)
        .write(path)
        .compare()

    attr
        task = ForeignKeyField(Task, related_name='task')
        watch_date = DateTimeField()
        title = CharField(null=True)
        author = CharField(null=True)
        content = TextField(null=True)
        metadata = TextField(null=True)
        topic = TextField(null=True)
        question = TextField(null=True)
        comment = TextField(null=True)





#### Cleaner

处理, 清理文本的规则



Exporter

根据 Template 转换为 rss, epub, pdf, 以及 github diff 预览

图片转存也算在 Template 里

In [ ]:
描述

以更加简洁清晰的排版阅读知乎. (这个需求有简悦)

用户指定一个知乎网站上的问题, zhihumark 抓取相应的回答内容, 以 markdown 文档的形式提供给用户阅读和下载. (提供 md 下载没用)





功能

可以自由定制想抓取的内容, 这包括: 抓取指定答主/收藏夹名/问题/话题下的回答, 抓取赞同数量超过N的回答.
可以设置黑名单, 不抓取特定的问题或者答主.
抓取评论, 以会话的形式组织起来, 让上下文相关的评论出现在一起, 尽量滤除没有价值的评论.
将收集到的一系列文档保存为epub或者pdf, 或者批量下载 markdown 文档, 以便离线阅读.
制作成 feed 以便放入 rss 阅读器, 获取实时更新.


观念

我们也不显示其他人搜索了什么关键字, 获取了什么页面, 下载了哪些电子书, 看到什么取决于你

我们相信评论是回答(或文章)可以分割的一部分, 许多评论很水, 有些很有价值.

相信评论区是对回答的重要补充, 也有随回答保留的必要, 但需处理评论区良莠不齐, 引用回复混乱的问题.



页面设计

单页面网站应用, 基本上类似搜索引擎的风格和排版.

输入框 - 接受输入ID或URL 答主/收藏夹名/问题名/话题名

选项 - 保留最前的n个回答?
选项 - 赞同数至少是?
选项 - 保留评论? 评论对话中至少有多少 likes
屏蔽 - 不感兴趣的话题, 问题, 答主
返回 - markdown文件列表

批量下载按钮

In [ ]:
import tools
tools.generate_ascii_title('flask')

### Test Collector

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from collector import Collector

col = Collector(project_path='D:\DataStore\The North Remembers - Science')


col = Collector.create_project(r'D:\DataStore\Test Collector8')
# col = Collector(project_path='/project')
# topic_id = 19644231 # 古建筑
# topic_id = 19582176 # 建筑设计
# topic_id = 19573393 # 建筑史
# topic_id = 19568972 # 建筑学
col.report()

create new project D:\DataStore\Test Collector8 and committed
report <Collector #3009843698880 path=`D:\DataStore\Test Collector8`> 
---------------------
git log: 
    5a501f2 init TNR project
---------------------


In [6]:
urls = '''
https://zhuanlan.zhihu.com/learningtheory #动物行为和学习
https://zhuanlan.zhihu.com/memento        #Memento
https://zhuanlan.zhihu.com/bioandchill    #老司机的生物学课堂
'''.strip().splitlines()

col.create_watcher(urls=[l.split(' ')[0]for l in urls], 
                   folder='知乎生物学专栏合集', 
                   parent_folder='Science',
                   lister_option={'zhihu_min_voteup': 1, 'limit': 2})
print(col)

generate_tip get 动物行为和学习
generate_tip get Memento
generate_tip get 老司机的生物学课堂
Watcher path D:\DataStore\Test Collector8\Science\知乎生物学专栏合集 created
Watcher.remember committed: "create Watcher folder 知乎生物学专栏合集"

<Collector #3009843698880 path=`D:\DataStore\Test Collector8`>


In [7]:
col.watching('知乎生物学专栏合集')

config `D:\DataStore\Test Collector8\Science\知乎生物学专栏合集/.task.yaml` loaded: 
  lister default option: {'enabled': True, 'max_cycle': '30days', 'min_cycle': '12hours', 'weight': 0.5, 'limit': 1, 'zhihu_min_voteup': 1}
  page default option: {'enabled': True, 'max_cycle': '180days', 'min_cycle': '45days', 'weight': 0.5}
  found 3 lister tasks in config:
    lister_task: 知乎专栏 - 动物行为和学习
    lister_task: 知乎专栏 - Memento
    lister_task: 知乎专栏 - 老司机的生物学课堂
  found 0 page tasks in config:

watcher add 3 tasks: {'new+prepare': 3}
watcher add 0 tasks: {}

  ↓ start_watching_once for
  <Watcher #3009843698264>
      from "D:\DataStore\Test Collector8\Science\知乎生物学专栏合集", 3 tasks
    
watching listers... should fetch 3 lister tasks

request_ZhihuColumnLister column_id, learningtheory
detect https://zhuanlan.zhihu.com/p/24675651 狗有情景记忆吗？ - 大哉问
Task.run detect new tasks done: 1 tasks
watcher add 1 tasks: {'new+prepare': 1}
lister task done (1/3): 
<Task #3009839215728> https://zhuanlan.zhihu.com/learnin

In [ ]:
# col = Collector.create_project(r'D:\DataStore\Test Collector13')

选项分为 "global_default_option:" 
        '''
        watcher_config:
          folder_path:
            default_option:
        '''
有 / 时自动创建各级路径 如 "科学/生物学:"

lister_task: 如果时 字符串, 视为简写 url


configtext = '''


global_default_option:
  {}


watcher_config:
  科学/生物学:
    default_option:
      lister:
        enabled: true
        max_cycle: 30days
        min_cycle: 12hours
        weight: 0.5
        limit: 200
        zhihu_min_voteup: 1000
      page:
        enabled: true
        max_cycle: 180days
        min_cycle: 45days
        weight: 0.5
    lister_task:
        - 'https://zhuanlan.zhihu.com/learningtheory #动物行为和学习'
        - 'https://zhuanlan.zhihu.com/memento        #Memento'
        - 'https://zhuanlan.zhihu.com/bioandchill    #老司机的生物学课堂'
        
  科学/<auto>:
      default_option: 
        lister:     
          enabled: true
          max_cycle: 7days
          min_cycle: 12hours
          weight: 0.5
          limit: 200
          zhihu_min_voteup: 500
          banned_keywords: 情感, 调查类问题, 有哪些, 文艺表达, 文艺的表达
        page: 
          enabled: true
          max_cycle: 180days
          min_cycle: 15days
          weight: 0.5
          
      lister_task:
        - 'https://zhuanlan.zhihu.com/learningtheory #动物行为和学习'
        - 'https://zhuanlan.zhihu.com/memento        #Memento'
        - 'https://zhuanlan.zhihu.com/bioandchill    #老司机的生物学课堂'
      

      
'''





config = tools.yaml_loads(configtext)
col.create_multiple_watchers(config=config)


In [ ]:


urls = '''
https://zhuanlan.zhihu.com/memento Memento
'''.strip().splitlines()

col.create_watcher(urls=[l.split(' ')[0]for l in urls], 
                   tips=[l.split(' ')[1]for l in urls], 
                   folder='生物学', 
                   parent_folder='科学',
                   lister_option={'zhihu_min_voteup': 100, 'limit': 3})
print(col)

urls = '''
https://zhuanlan.zhihu.com/c_116602381 魔鬼眼中的自然界
'''.strip().splitlines()
col.create_watcher(urls=[l.split(' ')[0]for l in urls], 
                   # tips=[l.split(' ')[1]for l in urls], 
                   # folder=None, 
                   parent_folder='科学',
                   lister_option={'zhihu_min_voteup': 1000, 'limit': 5})
print(col)

In [ ]:
urls = [
  'https://www.zhihu.com/topic/19577481/top-answers/', # 量子物理
  'https://www.zhihu.com/topic/19803067/top-answers/', # 量子通信
  'https://www.zhihu.com/topic/19938157/top-answers/', # 量子计算机
]

col.create_watcher(urls, lister_option={'zhihu_min_voteup': 1000, 'limit': 8})
str(col)

In [ ]:
col.report()

In [ ]:
col.watching('知乎话题 - 量子物理, 量子通讯, 量子计算机')

In [ ]:
# from watcher import Watcher

# w = Watcher(col.find_watcher_path('知乎专栏 - 魔鬼眼中的自然界'))
# for l in w.watch():
#   print(l)

In [ ]:
urls = '''
https://zhuanlan.zhihu.com/learningtheory #动物行为和学习
https://zhuanlan.zhihu.com/memento        #Memento
https://zhuanlan.zhihu.com/bioandchill    #老司机的生物学课堂
'''.strip().splitlines()

col.create_watcher(urls=[l.split(' ')[0]for l in urls], 
                   folder='知乎生物学专栏合集', 
                   parent_folder='Science',
                   lister_option={'zhihu_min_voteup': 1})
print(col)

In [ ]:
# url = 'https://zhuanlan.zhihu.com/wision'
# tip = '智堡研究所'
# # col.add(url, tip, lister_option=None, page_option=None)
# col.add(url, tip)
# str(col)

In [ ]:
urls = [
  'https://www.zhihu.com/topic/19577481/top-answers/', # 量子物理
  'https://www.zhihu.com/topic/19803067/top-answers/', # 量子通信
  'https://www.zhihu.com/topic/19938157/top-answers/', # 量子计算机
]

col.create_watcher(urls, lister_option={'zhihu_min_voteup': 1000})
str(col)

In [ ]:
# collector.add(weixin_page_url)
# collector.add(zhihu_column_url)
# collector.add(zhihu_anwser_id=xxx)
# collector.add(zhihu_author_id=xxx)
# collector.add(column_id=xxx)

# collector.save(file=xxx)
# ...
# collector.rescan()
# collector.save(file=xxx)
# collector = Collector.load(file=xxx.yaml)

### Test Watcher

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from watcher import Watcher


# w = Watcher(r'D:\DataStore\The North Remembers - Social\篝火旁的故事')
# w = Watcher(r'D:\DataStore\The North Remembers - Social\沉思的托克维尔')
# w = Watcher(r'D:\DataStore\The North Remembers - Science\何为现实？拉普拉斯之妖与薛定谔猫之决战')

# w = Watcher(r'D:\DataStore\The North Remembers - Art\知乎话题 - 古建筑, 建筑设计, 建筑史, 建筑学')

w = Watcher(r'D:\DataStore\test\机票')
# w = Watcher(r'D:\DataStore\The North Remembers - Game\旗舰评论——战略航空军元帅的旗舰')
# w = Watcher(r'D:\DataStore\test\经济')
# w = Watcher(r'D:\DataStore\The North Remembers - Social\何瑫的写作工坊')
# w = Watcher(r'D:\DataStore\The North Remembers - Science/知乎话题 - 量子物理, 量子通讯, 量子计算机')


In [ ]:
for commit_data in w.watch():
  print(commit_data)

In [ ]:
u = 'https://www.zhihu.com/question/19848582/answer/18231808'
question = parse_question(u)
answer = parse_answer(u)
# try:
#   title = question.title
#   detail = question.detail
# except GetDataErrorException:
#   # zhihu_oauth.exception.GetDataErrorException:
#   # A error happened when get data: 问题不存在
#   raise QuestionDeleteError(msg='问题已删除', url=url)


In [ ]:
question.title

In [ ]:
# 为什么 9·11 恐怖袭击中双子塔在一段时间的燃烧后倒塌了？
fetch_zhihu_answer('https://www.zhihu.com/question/19848582/answer/18231808')

In [ ]:
d = _
print(d.get('comments'))

### Test Generate Feed 

In [ ]:
from collector import Collector
from page import Page
col = Collector.create_project(r'D:\DataStore\Test Collector2')
col.report()

In [ ]:
col.generate_feed('知乎话题 - 量子物理, 量子通讯, 量子计算机', limit=1, site='http://xxxx.com/')


In [ ]:
page = Page.load(r'D:\DataStore\Test Collector2\知乎话题 - 量子物理, 量子通讯, 量子计算机\中科院在铁基超导体中发现天使粒子魅影马约拉纳费米子模，什么是马约拉纳费米子模？其发现有何意义？ - 中科院之声的回答.md')

In [ ]:
page.fetch_date

In [ ]:
resp.response

### Test Zhihu Fetch Error

In [ ]:
from fetcher_api.zhihu import fetch_zhihu_answer
from fetcher_api.zhihu import parse_answer
from fetcher_api.zhihu import parse_question

### Fetch comments MaxRetryError

In [ ]:
from pprint import pprint
from fetcher_api.zhihu import parse_question
from fetcher_api.zhihu import parse_answer
from fetcher_api.zhihu import zhihu_detect_with_client
from fetcher_api.zhihu import zhihu_detect
from fetcher_api.zhihu import get_comments_api_v4
from fetcher_api.zhihu import get_valuable_conversations_api_v4

### generate token

In [ ]:
from fetcher import generate_zhihu_token
generate_zhihu_token()

In [ ]:
client = zhihu.client

In [ ]:
c = client.collection(133030984)
int("133030984")

In [ ]:
for ans in c.answers:
  print(ans.author.name)

In [ ]:
from zhihu_oauth import ZhihuClient

TOKEN_FILE = 'token.pkl'
client = ZhihuClient()
client.load_token(TOKEN_FILE)


In [ ]:
page = client.from_url('https://zhuanlan.zhihu.com/p/37509962')

In [ ]:
from fetcher_api.zhihu import fetch_zhihu_article

In [ ]:
fetch_zhihu_article('https://zhuanlan.zhihu.com/p/37509962')

In [ ]:
import os
os.path.basename('D:\Coding\TheNorthRemembers\page.py')

In [ ]:
os.path.__file__

In [ ]:
import wechatsogou

In [ ]:
ws_api = wechatsogou.WechatSogouAPI()

In [ ]:
ws_api.get_gzh_info('南航青年志愿者')

In [ ]:
list(ws_api.search_article('马前卒工作室'))

In [ ]:
topic_id = 11
f'generate_tip {topic_id}'

### Playground

In [ ]:
ret = fetch_zhihu_answer('https://www.zhihu.com/question/21567651/answer/56227075')

In [ ]:
ret['answer'].splitlines()

In [ ]:
u = 'https://zhuanlan.zhihu.com/p/30722439'

fetch_zhihu_article(u)

In [ ]:
from markdown import markdown
from mdx_gfm import GithubFlavoredMarkdownExtension



def to_html(full_text, cut=0):
  ''' 转换为 html 用于输出 RSS 等
      必须有 data['full_text'] 才能 to_html() '''
  if cut and len(full_text) > cut:
    full_text = full_text[:cut] + f' ... (共 {len(full_text)} 字)'

  html = markdown(full_text, output_format='html5', extensions=[GithubFlavoredMarkdownExtension()])
  return html

In [ ]:
sys.argv

In [ ]:
import tools
tools.generate_ascii_title('design')

In [ ]:
urls = '''
ONES Piece (Zhihu)Design
https://zhuanlan.zhihu.com/onespiece

'''

urls = [u for u in urls.strip().splitlines() if u.startswith('http')]

In [ ]:
for u in urls:
  c = client.from_url(u)
  d = c.description.replace('\n', ' ')
  print(f'{c.title} - {c.articles_count} - {d}')
  print(u)


In [ ]:
import tools

In [ ]:

d  = {"tip": "[福利]坚果云+Eagle序列号，内部团购来了... - 酷coo豆"}
tools.yaml_saves(d)

In [ ]:

ban_topics=( )
type(ban_topics)

In [ ]:
print(tools.time_now())

In [ ]:
topic = client.from_url('https://www.zhihu.com/topic/19564001/')

In [ ]:
set('')

In [ ]:
banned_keywords = set()

any((key in 'ashfasfh') for key in banned_keywords)

In [ ]:
for i, answer in tools.enumer(topic.best_answers, first=30):
  topics = ' '.join(t.name for t in answer.question.topics)
  ratio = answer.voteup_count / answer.thanks_count
  result = f'''
  {answer.question.title} 赞{answer.voteup_count} 谢{answer.thanks_count} ({ratio:.2f})
  {topics}'''
  print(result)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
f = '《史记》记载孔子将原有的三千首有余的《诗经》删除，只剩三百零五篇。并说“诗三百，一言以蔽之，思无邪”可为什么还有《野有死麕》这样的略有点太过于开放、露骨的诗歌存在？ - kim vic的回答.md'

def filename(name): 
  # name = remove_invalid_char(self.metadata['title']) + '.md'
  enlen = lambda t: tools.encode_len(t, encode='utf-8')
  if  enlen(name) > 255:  
    # Linux 视为 ext4 格式, 需要 filename 不超过 255 字符
    parts = name.split(' ')
    long_part_index = parts.index(max(parts, key=enlen))
    limit = enlen(parts[long_part_index]) - (enlen(name) - 255)
    parts[long_part_index] = tools.truncate(parts[long_part_index], limit=limit, encode='utf-8')
    return ' '.join(parts)
  else:
    return name


filename(f), tools.encode_len(filename(f), 'utf-8')